In [4]:
# Aim of the exercise is to measure effectiveness of each marketing channel and create a budget allocator for the Markerting Team 
# Hence we would be going for explanatory model and not a predictive model 

In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

ModuleNotFoundError: No module named 'numpy'

In [ ]:
import seaborn as sns 
import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.stats.outliers_influence import variance_inflation_factor

from matplotlib.pyplot import figure

from math import ceil
import datetime

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import max_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_absolute_percentage_error

from datetime import date,timedelta


import warnings; warnings.simplefilter('ignore')


In [ ]:
def week_of_month(dt):
    """ Returns the week of the month for the specified date.
    """

    first_day = dt.replace(day=1)

    dom = dt.day
    adjusted_dom = dom + first_day.weekday()

    return int(ceil(adjusted_dom/7.0))

# Decomposition
def decompose(df_col,period,model):
    decomposition = sm.tsa.seasonal_decompose(df_col,period =12,model=model) 
    figure2 = decomposition.plot()
    figure(figsize=(16, 12), dpi=80)
    plt.show()

# Adfuller Test
def adftest(df_col):
    result = adfuller(df_col)
    print(f'ADF Statistic: {result[0]}')
    print(f'p-value: {result[1]}')
    print(f'Critical Values:')
    for key,val in result[4].items():
        print(f'{key} {val}')

# Plot ACF and PACF
def plot_apacf(df_col,lags):
    plot_acf(df_col, lags=lags)
    plt.show()
    plot_pacf(df_col,lags=lags)
    plt.show()
    
# Check VIF
def checkVIF(df,cols):
    vif_data = pd.DataFrame()
    X = df[cols]
    vif_data["feature"] = X.columns
  
    # calculating VIF for each feature
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]
    return vif_data

## Doing Log Transformations
def logTransform(df,cols):
    temp = df.copy()
    arr = []
    for j in cols:
        temp[f'ln_{j}'] = np.log(temp[j])
        arr.append(f'ln_{j}')
    return(temp,arr)


In [ ]:
# Fit a simple Linear regression 
# Input : Dataframe, Dependent Var, Independent Var, True/False for only positive Coeff, True/False for Plots
def LRfit(df,dv,iv,ng,plot,printstats):
    tt = df[df[dv].isna()==False]
    reg_nnls = LinearRegression(positive=ng)

    X = tt[iv]
    y = tt[dv]

    reg_nnls2 = reg_nnls.fit(X,y)

    y_pred = reg_nnls2.predict(X)
    
    vif = checkVIF(X,iv)
    
    

    run_year = tt['Date'].min()
    r2 = r2_score(y, y_pred)
    rmse = np.sqrt(mean_squared_error(y,y_pred))
    nmrse1 = rmse/y.std()
    nmrse2 = rmse/(y.max() - y.min())
    mae = mean_absolute_error(y,y_pred)
    mape = mean_absolute_percentage_error(y,y_pred)
    

    coeff_arr = []

    for q in reg_nnls2.coef_: 
        coeff_arr = np.insert(coeff_arr,len(coeff_arr),round(q,4))
    inter = reg_nnls2.intercept_
    r2 = r2_score(y, y_pred)
    
    if printstats ==True:
        print(vif)
        print(run_year)
        print(reg_nnls2.intercept_)
        print(f'Coeff : {coeff_arr}')
        print(f"R2 : {(r2)}")
        print(f"RMSE : {rmse}")
        print(f"NMRSE1 : {nmrse1}")
        print(f"NMRSE2 : {nmrse2}")
        print(f"MAE : {(mae)}")
        print(f"MAPE : {(mape)}")

    if plot == True:
        sns.lineplot(data=tt,x=range(len(X)),y=dv,label='1')
        sns.lineplot(x=range(len(y_pred)),y=y_pred,label='2')

    return(y_pred,run_year,dv,iv,r2,rmse,nmrse1,nmrse2,mae,mape,inter,coeff_arr,vif)


def convert_lr_fit_to_df(df, dv, iv):
    return pd.DataFrame([LRfit(df,dv,iv)], columns=['model_run_TF','DV','IV','R2','RMSE','NMRSE1','NMRSE2','MAE','MAPE','Intercept','Coeff','VIF'])


In [ ]:
# Fit a simple Ridge regression 
def RidgeFit(df,dv,iv,ng,st,norm,alph,plot):
    tt = df[df[dv].isna()==False]
    reg = linear_model.Ridge(alpha=alph,positive=ng,normalize=norm)

    X = tt[iv]
    y = tt[dv]

    reg2 = reg.fit(X,y)

    y_pred = reg2.predict(X)
    coeff = reg2.coef_
    
    inter= reg2.intercept_


    coeff_arr = []

    for q in coeff: 
        coeff_arr = np.insert(coeff_arr,len(coeff_arr),round(q,4))

    run_year = tt['Date'].min()
    r2 = r2_score(y, y_pred)
    rmse = np.sqrt(mean_squared_error(y,y_pred))
    nmrse1 = rmse/y.std()
    nmrse2 = rmse/(y.max() - y.min())
    mae = mean_absolute_error(y,y_pred)
    mape = mean_absolute_percentage_error(y,y_pred)

    vif = checkVIF(X,iv)
    

    if plot == True:
        sns.lineplot(data=tt,x=range(len(X)),y=dv,label='1')
        sns.lineplot(x=range(len(y_pred)),y=y_pred,label='2')
        print(vif)
        print(f'Intercept : {inter}')
        print(f'Coeff : {coeff_arr}')
        print(f'R2 : {r2}')
        print(f'RMSE : {rmse}')
        print(f'MAE : {mae}')
        print(f'MAPE : {mape}')

    return(st,y_pred,run_year,dv,iv,r2,rmse,nmrse1,nmrse2,mae,mape,inter,coeff_arr,vif)

def convert_ridge_fit_to_df(df, dv, iv,ng,st,norm,alph):
    return pd.DataFrame([RidgeFit(df,dv,iv,ng,st,norm,alph,False)], columns=['base iv','y_pred','model_run_TF','DV','IV','R2','RMSE','NMRSE1','NMRSE2','MAE','MAPE','intercept','Coeff','VIF'])


In [ ]:
## Subset according to days
def dfTFdays(df,days):
    return(df[df['Date']>=[date.today()- timedelta(days=days)]*len(df)])

def spliceTF(df,days,enddt):
     return(df[(df['Date']<=enddt) & (df['Date']>=[pd.to_datetime(enddt)- timedelta(days=days)]*len(df))])

In [ ]:
# read input file
data = pd.read_csv('Burger_Sales.csv')
print(f'Lenght of Data Frame : {len(data)}')
print(f'Minimum Date : {data[data.columns[0]].min()}')
print(f'Minimum Date : {data[data.columns[0]].max()}')
print(f'Columns : {data.columns}')

In [ ]:
data

In [ ]:
corr = data_sel[['burger_sales', 'Instagram_Cost', 'Bing_Cost', 'Facebook_Cost',
       'Google_Cost', 'Pinterest_Cost', 'Tiktok Cost']].corr()

sns.heatmap(corr, 
        xticklabels=corr.columns,
        yticklabels=corr.columns)

# It seems our dataset has a lot of correlation in between the spends 
# Note: Marketing spends are generally increased/decreased across channels together


In [ ]:
# overall Burger Sales Time Series
plt.plot(data['burger_sales'])
plt.show()

# Q-Q Plot : To check for normality
sm.qqplot(data['burger_sales'])
plt.show()


In [ ]:
sns.kdeplot(data['burger_sales'])
# Two different distribution of Sales ; This might be due to new marketing channels coming into play  

In [ ]:
# Considering the time before 2021
date_cond = data['Date'] < '2021-01-09'
plt.plot(data[date_cond][:]['burger_sales'])
plt.show()

sns.kdeplot(data[date_cond]['burger_sales'])
plt.show()

# Q-Q Plot : To check for normality
sm.qqplot(data[date_cond]['burger_sales'])
plt.show()

In [ ]:
# Time period after 2021 
# Let us use this more recent data to model the relationship between sales and marketing spends

date_cond = data['Date'] > '2021-01-08'
plt.plot(data[date_cond][:]['burger_sales'])
plt.show()

sns.kdeplot(data[date_cond]['burger_sales'])
plt.show()

# Q-Q Plot : To check for normality
sm.qqplot(data[date_cond]['burger_sales'])
plt.show()

In [ ]:
data_sel = data[date_cond]
decompose(data_sel['burger_sales'],12,'additive')
decompose(data_sel['burger_sales'],12,'multiplicative')

# Seems like a multiplicative model would be better as the residuals are mode stable 

In [ ]:
plot_apacf(data_sel['burger_sales'],20)

In [ ]:
# Lets check for Multicollinearity 

checkVIF(data_sel,['burger_sales', 'Instagram_Cost', 'Bing_Cost', 'Facebook_Cost','Google_Cost', 'Pinterest_Cost', 'Tiktok Cost'])

In [ ]:
# Replacing 0 by 0.01 in any spends , so that a log transformation can be taken
for q in data_sel.columns[1:]:
    print(q)
    data_sel[q] = np.where(data_sel[q]==0,0.01,data_sel[q])
    data_sel[f'ln_{q}'] = np.log(data_sel[q])
    

# We will create some seasonality features eg : year, month, day, dayofweek, weekofmonth

data_sel['Date'] = pd.to_datetime(data_sel['Date'])
data_sel['year'] = data_sel['Date'].dt.year
data_sel['month'] = data_sel['Date'].dt.month
data_sel['day'] = data_sel['Date'].dt.day
data_sel['dayofWeek'] = data_sel['Date'].dt.dayofweek
data_sel['dayofWeek']  = data_sel['dayofWeek']  + 1
data_sel['weekOfMonth'] = data_sel['Date'].apply(lambda x: week_of_month(x))
data_sel['weekofYear'] = data_sel.apply(lambda x: datetime.date(x.year, x.month, x.day).isocalendar()[1], axis=1)


data_sel.head()

In [ ]:
plt.plot(data_sel[:]['ln_burger_sales'])
plt.show()

sns.kdeplot(data_sel['ln_burger_sales'])
plt.show()

# Q-Q Plot : To check for normality
sm.qqplot(data_sel['burger_sales'])
plt.show()

In [ ]:
# Lets check for Multicollinearity 

checkVIF(data_sel,['burger_sales', 'Instagram_Cost', 'Bing_Cost', 'Facebook_Cost','Google_Cost', 'Pinterest_Cost', 'Tiktok Cost'])

In [ ]:
checkVIF(data_sel,['ln_burger_sales', 'ln_Instagram_Cost', 'ln_Bing_Cost', 'ln_Facebook_Cost','ln_Google_Cost', 'ln_Pinterest_Cost', 'ln_Tiktok Cost'])

In [ ]:
checkVIF(data_sel,['ln_burger_sales', 'Instagram_Cost', 'Bing_Cost', 'Facebook_Cost','Google_Cost', 'Pinterest_Cost', 'Tiktok Cost'])

In [ ]:
checkVIF(data_sel,['burger_sales','month','dayofWeek','weekOfMonth'])

In [ ]:
# We will go with 2 models. 
# First is a base model without spends 
# Second Model will include the base sales and the Marketing Spends

In [ ]:
ctrl_arr = ['month','dayofWeek','weekOfMonth']
iv_arr = ['Instagram_Cost', 'Bing_Cost', 'Facebook_Cost','Google_Cost', 'Pinterest_Cost', 'Tiktok Cost']
iv_ln_arr = ['ln_Instagram_Cost', 'ln_Bing_Cost', 'ln_Facebook_Cost','ln_Google_Cost', 'ln_Pinterest_Cost', 'ln_Tiktok Cost']


In [ ]:
y_pred,run_year,dv,iv,r2,rmse,nmrse1,nmrse2,mae,mape,inter,coeff_arr,vif = LRfit(data_sel,'ln_burger_sales',ctrl_arr,False,True,False)
data_sel['base_ln'] = y_pred
data_sel['base_y'] = np.exp(y_pred)

base_model_intercept = inter
base_model_coeffs = coeff_arr

In [ ]:
from sklearn import linear_model
st,y_pred,run_year,dv,iv,r2,rmse,nmrse1,nmrse2,mae,mape,inter,coeff_arr,vif = RidgeFit(data_sel,'ln_burger_sales',np.append(['base_ln'],iv_ln_arr),True,'test',False,1,True)

# We cannot have 0 as coefficient for any of the channels as it means there is 0 contributio from that channel 
# Lets try for different Time periods 

In [ ]:
# we will iterate every 15 days and fit a sales, ln_sales with two different independent variables 
# our objective is to find a time period for each we get a positive coefficient for each channel 
# each row at the end is a model fitted with different metrics

ChampChallenger = pd.DataFrame(columns=['base iv','model_run_TF','DV','IV','R2','RMSE','NMRSE1','NMRSE2','MAE','MAPE','intercept','Coeff','VIF'])
testcp = data_sel.copy()

## Iterating over 1 to 3 years, with 1 month increment
for p in range(180,1080,15):
    
    df = spliceTF(testcp,p,'2022-05-31')
    df,arr1 = logTransform(df,['burger_sales'])
    df,arr1  = logTransform(df,iv_arr)

    y_pred,run_year,dv,iv,r2,rmse,nmrse1,nmrse2,mae,mape,inter,coeff_arr,vif = LRfit(data_sel,'ln_burger_sales',ctrl_arr,False,False,False)
    data_sel['base_ln'] = y_pred
    data_sel['base_y'] = np.exp(y_pred)

    iv_arr = ['base_y','Instagram_Cost', 'Bing_Cost', 'Facebook_Cost','Google_Cost', 'Pinterest_Cost', 'Tiktok Cost']
    iv_ln_arr = ['base_ln','ln_Instagram_Cost', 'ln_Bing_Cost', 'ln_Facebook_Cost','ln_Google_Cost', 'ln_Pinterest_Cost', 'ln_Tiktok Cost']

   
    dvt = ['burger_sales','ln_burger_sales']

    for ivt in [iv_ln_arr]:
        for dvtp in dvt:
            if (convert_ridge_fit_to_df(df=df, dv=dvtp, iv=ivt,ng=True,st=f' na',norm=False,alph=0.9).iloc[0]['Coeff'].min()<=0):
                continue
            else:
                ChampChallenger = ChampChallenger.append(convert_ridge_fit_to_df(df=df, dv=dvtp, iv=ivt,ng=True,st=f'alpha:{p}',norm=False,alph=0.9))

ChampChallenger = ChampChallenger.sort_values(by='MAPE',ascending=True)


ChampChallenger

In [ ]:
# we will take the top row as model 
from sklearn import linear_model

testcp = data_sel.copy()
testcp = spliceTF(testcp,195,'2022-05-31')

y_pred,run_year,dv,iv,r2,rmse,nmrse1,nmrse2,mae,mape,inter,coeff_arr,vif = LRfit(testcp,'ln_burger_sales',ctrl_arr,False,False,True)
testcp['base_ln'] = y_pred
testcp['base_y'] = np.exp(y_pred)
st,y_pred,run_year,dv,iv,r2,rmse,nmrse1,nmrse2,mae,mape,inter,coeff_arr,vif = RidgeFit(testcp,'ln_burger_sales',iv_ln_arr,True,'test',False,1,True)

In [ ]:
"""
Our final model : 

1. Control Model : 
Intercept : 9.29974879006081
Coeff : [-0.0325 -0.0513  0.0087]

2. MMM Model : 
Intercept : 3.379554614274598
Coeff : [0.0692 0.0043 0.0013 0.1646 0.2793 0.0289 0.002 ]

"""

In [ ]:
# storing final model coeffs and intercepts
base_intecept = 9.29974879006081
base_coeffs = [-0.0325, -0.0513,  0.0087]

mmm_intercept = 3.379554614274598
mmm_coeff = [0.0692, 0.0043, 0.0013, 0.1646, 0.2793, 0.0289, 0.002 ]


In [ ]:
# calculating channel contributions and ROAS
df = testcp.copy()
df.columns

coeffs = mmm_coeff
df['pred_ln'] = y_pred
df['pred_y'] = np.exp(df['pred_ln'])


df['baseline'] = np.exp(mmm_intercept)*(df['base_y']**coeffs[0])
df['multiplicative_pred']= df['baseline']* df['Instagram_Cost']**(coeffs[1]) * (df['Bing_Cost']**(coeffs[2])) * (df['Facebook_Cost']**(coeffs[3])) * (df['Google_Cost']**(coeffs[4])) * (df['Pinterest_Cost']**(coeffs[5])) * (df['Tiktok Cost']**(coeffs[6]))

# Calculating Contributions
df['Instagram_Contri'] = df['multiplicative_pred']*(1-1/df['Instagram_Cost']**(coeffs[1]))
df['Bing_Contri'] = df['multiplicative_pred']*(1-1/df['Bing_Cost']**(coeffs[2]))
df['Facebook_Contri'] = df['multiplicative_pred']*(1-1/df['Facebook_Cost']**(coeffs[3]))
df['Google_Contri'] = df['multiplicative_pred']*(1-1/df['Google_Cost']**(coeffs[4]))
df['Pinterest_Contri'] = df['multiplicative_pred']*(1-1/df['Pinterest_Cost']**(coeffs[5]))
df['Tiktok_Contri'] = df['multiplicative_pred']*(1-1/df['Tiktok Cost']**(coeffs[6]))

df['Total_Contri'] = df['Instagram_Contri'] + df['Bing_Contri'] + df['Facebook_Contri'] + df['Google_Contri'] + df['Pinterest_Contri'] + df['Tiktok_Contri']
df['Actual_Contri'] = df['multiplicative_pred'] - df['baseline']

# Adjusting the Contributions
df['Instagram_Adj_Contri'] = df['Instagram_Contri']*(1-(df['Total_Contri']-df['Actual_Contri'])/(df['Total_Contri']))
df['Bing_Adj_Contri'] = df['Bing_Contri']*(1-(df['Total_Contri']-df['Actual_Contri'])/(df['Total_Contri']))
df['Facebook_Adj_Contri'] = df['Facebook_Contri']*(1-(df['Total_Contri']-df['Actual_Contri'])/(df['Total_Contri']))
df['Google_Adj_Contri'] = df['Google_Contri']*(1-(df['Total_Contri']-df['Actual_Contri'])/(df['Total_Contri']))
df['Pinterest_Adj_Contri'] = df['Pinterest_Contri']*(1-(df['Total_Contri']-df['Actual_Contri'])/(df['Total_Contri']))
df['Tiktok_Adj_Contri'] = df['Tiktok_Contri']*(1-(df['Total_Contri']-df['Actual_Contri'])/(df['Total_Contri']))


df['Test_Contri'] = df['baseline'] + df['Instagram_Adj_Contri'] + df['Bing_Adj_Contri']  + df['Facebook_Adj_Contri']  + df['Google_Adj_Contri'] + df['Pinterest_Adj_Contri'] + df['Tiktok_Adj_Contri']

# ROAS = Adjusted Contribution / Cost 
df['Instagram ROAS'] = df['Instagram_Adj_Contri'] / df['Instagram_Cost']
df['Bing ROAS'] = df['Bing_Adj_Contri'] / df['Bing_Cost']
df['Facebook ROAS'] = df['Facebook_Adj_Contri'] / df['Facebook_Cost']
df['Google ROAS'] = df['Google_Adj_Contri'] / df['Google_Cost']
df['Pinterest ROAS'] = df['Pinterest_Adj_Contri'] / df['Pinterest_Cost']
df['Tiktok ROAS'] = df['Tiktok_Adj_Contri'] / df['Tiktok Cost']

In [ ]:
# Contribution Charts
sns.scatterplot(data=df,x='Instagram_Cost',y='Instagram_Adj_Contri',label='Instagram')
sns.scatterplot(data=df,x='Bing_Cost',y='Bing_Adj_Contri',label='Bing')
sns.scatterplot(data=df,x='Tiktok Cost',y='Tiktok_Adj_Contri',label='Tiktok')


In [ ]:
sns.scatterplot(data=df,x='Facebook_Cost',y='Facebook_Adj_Contri',label='Facebook')
sns.scatterplot(data=df,x='Google_Cost',y='Google_Adj_Contri',label='Google')


In [ ]:
sns.scatterplot(data=df,x='Facebook_Cost',y='Facebook_Adj_Contri',label='Facebook')
sns.scatterplot(data=df,x='Google_Cost',y='Google_Adj_Contri',label='Google')

sns.scatterplot(data=df,x='Instagram_Cost',y='Instagram_Adj_Contri',label='Instagram')
sns.scatterplot(data=df,x='Bing_Cost',y='Bing_Adj_Contri',label='Bing')
sns.scatterplot(data=df,x='Tiktok Cost',y='Tiktok_Adj_Contri',label='Tiktok')


In [ ]:
sns.scatterplot(data=df,x='Pinterest_Cost',y='Pinterest_Adj_Contri',label='Pinterest')